# Turkish Diacritisation | YZV 405E NLP Term Project

Author: Bora Boyacıoğlu

Student ID: 150200310

## Step 2: Training

In [17]:
import pickle as pkl

import torch
import torch.nn as nn
import torch.optim as optim

from utils.model import Seq2Seq

In [ ]:
%load_ext autoreload
%autoreload 2

### Reloading the Processed Data

In [7]:
# Load the train dataset.
train_dataset = pkl.load(open('data/train_data.pkl', 'rb'))

# Load the test dataset.
test_dataset = pkl.load(open('data/test_data.pkl', 'rb'))

### Defining the Training Loop

TypeError: DiacritizationModel.__init__() missing 4 required positional arguments: 'input_size', 'hidden_size', 'num_layers', and 'output_size'

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, (src, trg) in enumerate(iterator):
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)